In [5]:
!pip install tensorflow

In [6]:
!pip install tflearn

     ---------------------------------------- 0.0/107.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/107.3 kB ? eta -:--:--
     ------- -------------------------------- 20.5/107.3 kB ? eta -:--:--
     -------------- ---------------------- 41.0/107.3 kB 653.6 kB/s eta 0:00:01
     ---------------------------------- - 102.4/107.3 kB 737.3 kB/s eta 0:00:01
     ------------------------------------ 107.3/107.3 kB 776.1 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127291 sha256=183875fcbeed98c98652346877b3323e5ebd4c3aecfbb3beff9c29d7f2a3e1b1
  Stored in directory: c:\users\aran\appdata\local\pip\cache\wheels\2e\5f\cd\ffa06b65baff30574958318ce36ef9ef7af1ef3745dadaf420
Successfully built tflearn


In [7]:
!pip install tqdm

In [21]:
import cv2
import numpy as np
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt

%matplotlib inline


In [22]:
TRAIN_DIR = 'd:/picture/train'
TEST_DIR = 'd:/picture/test'
IMG_SIZE = 50
LR = 1e-3

MODEL_NAME = 'dogs-vs-cats-convnet'

def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat': 
        return [1, 0]
    elif word_label == 'dog': 
        return [0, 1]


In [23]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR, img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
        training_data.append([np.array(img), np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

def create_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR, img)
        img_num = img.split('.')[0]
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
        testing_data.append([np.array(img), img_num])
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    return testing_data


In [24]:
train_data = create_train_data()
test_data = create_test_data()

PermissionError: [WinError 21] 裝置未就緒。: 'd:/picture/train'

In [ ]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression


In [ ]:
convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')

convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log', tensorboard_verbose=0)


In [ ]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

train = train_data[:-8000]
test = train_data[-8000:]

X = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_y = [i[1] for i in test]

model.fit({'input': X}, {'targets': Y}, n_epoch=5, 
          validation_set=({'input': test_x}, {'targets': test_y}), 
          snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

model.save(MODEL_NAME)


In [ ]:
d = test_data[0]
img_data, img_num = d
data = img_data.reshape(IMG_SIZE, IMG_SIZE, 1)
prediction = model.predict([data])[0]

fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)
ax.imshow(img_data, cmap="gray")

print('Cat Possibility: {:5.2f}% \nDog Possibility: {:5.2f}%'.format(prediction[0]*100, prediction[1]*100))


In [ ]:
fig = plt.figure(figsize=(16, 12))
for num, data in enumerate(test_data[:16]):
    img_num = data[1]
    img_data = data[0]
    y = fig.add_subplot(4, 4, num + 1)
    orig = img_data
    data = img_data.reshape(IMG_SIZE, IMG_SIZE, 1)
    model_out = model.predict([data])[0]

    if np.argmax(model_out) == 1:
        str_label = 'Dog'
    else:
        str_label = 'Cat'

    y.imshow(orig, cmap='gray')
    plt.title(str_label)
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
plt.show()
